In [1]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc,jieba
from gensim.models import Word2Vec
from sklearn.model_selection import KFold

In [2]:
# train_data = []
# train_data_dict = {}
# with open('data/raw_data/train.json','r') as f:
#     for line in f:
#         raw = json.loads(line)
#         train_data.append(raw)
#         train_data_dict[raw['text_id']] = raw
# # 读取知识库
# kb_data = []
# kb = {}
# with open('data/raw_data/kb_data','r') as f:
#     for line in f:
#         item = json.loads(line)
#         kb[item['subject_id']] = item
#         kb_data.append(item)


In [3]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item

name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [4]:
"""
TODO:
预测type,
引入tfidf
"""

'\nTODO:\n预测type,\n引入tfidf\n'

In [5]:
def get_common_word(a, b):
        common_count = 0
        for word in a:
            if word in b:
                common_count += 1
        return common_count
def cal_similarity(params):
    index,sentence_a,sentence_b = params
    la = len(sentence_a)
    lb = len(sentence_b)

    char_a = [c for c in sentence_a]
    char_b = [c for c in sentence_b]
    word_a = jieba.lcut_for_search(sentence_a)
    word_b = jieba.lcut_for_search(sentence_b)
    bigram_a = [sentence_a[i:i + 2] for i in range(len(sentence_a) - 1)]
    bigram_b = [sentence_b[i:i + 2] for i in range(len(sentence_b) - 1)]
    trigram_a = [sentence_a[i:i + 3] for i in range(len(sentence_a) - 2)]
    trigram_b = [sentence_b[i:i + 3] for i in range(len(sentence_b) - 2)]

    common_word, common_word_score = 0, 0
    for w in word_b:
        if w in word_a:
            common_word += 1
            common_word_score += 1

    common_c = get_common_word(char_a, char_b)
    common_bigram = get_common_word(bigram_a, bigram_b)
    common_trigram = get_common_word(trigram_a, trigram_b)

    max_len = max(la, lb)
    min_len = min(la, lb)
    if max_len==0:
        max_len=0.01
    if min_len==0:
        min_len = 0.01
    numerical_feats = [max_len, min_len, common_word, common_word_score,
                                common_word / min_len, common_word / max_len,
                                common_word_score / min_len, common_word_score / max_len,
                                common_c / min_len, common_c / max_len,
                                common_bigram / min_len, common_bigram / max_len,
                                common_trigram / min_len, common_trigram / max_len,
                                common_word_score - common_word]
    return index, numerical_feats

In [6]:
def is_works(mention,text):
    if '《'+mention in text or mention+'》' in text:
        return 1
    elif '【'+mention+'】' in text:
        return 1
    elif '['+mention+']' in text:
        return 1
    else:
        return 0

In [7]:
# 书名号纠错

In [8]:
# 提取candidate候选特征
kb_column = []
train_column = []
text_id = []
label_colum = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
mention_end = []
numerical_f = []
is_eng = []
create_works = []
end_with_point = []
end_with_line = []
end_with_line2 = []
end_with_shuminghao = []
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        if m['kb_id'] != 'nil':
            name_list = [kb[m['kb_id']]['subject']]+kb[m['kb_id']]['alias']
            if m['mention'] not in name_list:
                    continue

        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['predicate']

                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            if m_candidate_id==m['kb_id']:
                label = 1
            else:
                label = 0
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            text_id.append(s['text_id'])
            label_colum.append(label)
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            candidate_name_list = candidate_detail['alias']+[candidate_detail['subject']]
            num_alias.append(len(set(candidate_name_list)))
            m_id_list.append(m_id)
            num_candidate.append(len(candidate_ids))
            equal_subject.append(candidate_detail['subject']==m['mention'])
            
            shu_in_abstract = False
            for sub_alia in candidate_name_list:
                if '《'+sub_alia+'》' in candidate_abstract:
                    shu_in_abstract = True
            shuminghao.append(shu_in_abstract)
            
#             if langid.classify(m['mention'])[0]=='en':
#                 len_mention.append(len(m['mention'].split()))
#                 is_eng.append(1)
#             else:
#                 len_mention.append(len(m['mention']))
#                 is_eng.append(0)
            len_mention.append(len(m['mention']))
            mention_start.append(m['offset'])
            mention_end.append(len(s['text'])-int(m['offset'])-len(m['mention']))
            
            
            i = 0
            for item in m_list:
                if item in candi_text: i+=1
            entity_common.append(i)
            create_works.append(is_works(m['mention'],s['text']))
            end_with_point.append('.' == s['text'][-1])
            end_with_line.append('-' in s['text'])
            end_with_line2.append('_' in s['text'])
            end_with_shuminghao.append('《' in s['text'])
        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['label'] = label_colum
data['num_attrs'] = num_attrs
data['num_abstract_words'] = num_abstract_words
data['num_alias'] = num_alias
data['m_id'] = m_id_list
data['num_candidates'] = num_candidate 
data['mention_equal_subject'] = equal_subject
data['shuminghao'] = shuminghao
data['entity_common']  = entity_common
data['len_mention'] = len_mention
data['mention_start'] = list(map(int,mention_start))
data['create_works'] = create_works
data['mention_end'] = mention_end
data['end_with_point'] = end_with_point
data['end_with_line'] = end_with_line
data['end_with_line2'] = end_with_line2
data['end_with_shuminghao'] = end_with_shuminghao
data['works_match'] = data.apply(lambda x: 1 if x.create_works==x.shuminghao else 0,axis=1)

100%|██████████| 90000/90000 [00:20<00:00, 4468.73it/s]


In [9]:
1==False

False

In [10]:
print(data.shape)


(1566680, 21)


In [11]:
# 提取candidate候选特征
numerical_f = []
i = 0
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        if m['kb_id'] != 'nil':
            name_list = [kb[m['kb_id']]['subject']]+kb[m['kb_id']]['alias']
            if m['mention'] not in name_list:
                    continue
        
        candidate_ids = name_id[m['mention']]
#         add_extra = 0 
#         if m['kb_id'] != 'nil':
#             name_list = [kb[m['kb_id']]['subject']]+kb[m['kb_id']]['alias']
            
#             if m['mention'] not in name_list:
#                 keep = False
#                 for name_c in ''.join(name_list):
#                     if name_c in m['mention']:
#                         #print(name_c,m['mention'])
#                         keep = True
#                         break
#                 if not keep:
#                     continue
#                 candidate_ids = name_id[m['mention']]+[m['kb_id']]
#                 add_extra = 1
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['predicate']
            numerical_f.append((i,candi_text,s['text']))
            i+=1
            #numerical_f.append(cal_similarity(candi_text,s['text']))


100%|██████████| 90000/90000 [00:10<00:00, 8664.93it/s]


In [12]:
import multiprocessing
p = multiprocessing.Pool(6)
res = p.map(cal_similarity, numerical_f)
res = sorted(res,key=lambda x:x[0],reverse=False)

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.766 seconds.
Prefix dict has been built succesfully.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.902 seconds.
Prefix dict has been built succesfully.
Dumping model to file cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.231 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.119 seconds.
Prefix dict has been built succesfully.
Loading model cost 0.875 seconds.
Prefix dict has been built succesfully.
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.194 seconds.
Prefix dict

In [13]:
nume_f = [x[1] for x in res]
numerical_f = np.array(nume_f)
for i in range(numerical_f.shape[1]):
    data['numerical_%d'%i] = numerical_f[:,i]

In [14]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,numerical_5,numerical_6,numerical_7,numerical_8,numerical_9,numerical_10,numerical_11,numerical_12,numerical_13,numerical_14
0,1,130287,南京南站,0,9,338,2,0,2,True,...,0.018648,0.421053,0.018648,4.210526,0.186480,1.736842,0.076923,0.842105,0.037296,0.0
1,1,311223,南京南站,1,17,447,3,0,2,True,...,0.012638,0.421053,0.012638,6.473684,0.194313,2.473684,0.074250,1.052632,0.031596,0.0
2,1,341096,高铁,1,10,239,1,1,1,True,...,0.009434,0.157895,0.009434,1.473684,0.088050,0.315789,0.018868,0.000000,0.000000,0.0
3,1,130287,南京南站,0,9,338,2,2,2,True,...,0.018648,0.421053,0.018648,4.210526,0.186480,1.736842,0.076923,0.842105,0.037296,0.0
4,1,311223,南京南站,1,17,447,3,2,2,True,...,0.012638,0.421053,0.012638,6.473684,0.194313,2.473684,0.074250,1.052632,0.031596,0.0


In [15]:
# f_columns = ['numerical_%d'%i for i in range(15)]

In [16]:
# data[f_columns].to_pickle('data/numerical_f.pkl')

In [17]:
# candidate　type的one-hot特征
data['type'] = data.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)
#data = pd.concat([data,pd.get_dummies(data['type'])],axis=1)

# target encoding
groupbytype = data.groupby('type',as_index=False)['label'].agg({
    'type_label_mean':'mean','type_label_count':'count'})
data = data.merge(groupbytype,on='type',how='left')

In [18]:
data.to_pickle('data/step1.pkl')

In [19]:
max_mid = data['m_id'].max()

In [20]:
# 载入测试数据

In [21]:
# # 戴安楠ner
# ner_result = pd.read_csv('result/el_dev_result(1).csv',sep='\t')
# ner_result['offset'] = ner_result['offset'].apply(lambda x:eval(x.lower()))
# ner_result['mention'] = ner_result['mention'].apply(lambda x:eval(x.lower()))
# ner_result = ner_result.rename(columns={'mention':'pred','offset':'pos'})


In [22]:
# my ner
ner_result = pd.read_pickle('result/ner_bert_result.pkl')
ner_result['text'] = ner_result['text'].apply(lambda x:''.join(x[1:-1]))
ner_result['pos'] = ner_result['pos'].apply(lambda x:[k[0]-1 for k in x])
ner_result['text_id'] = ner_result.index+1

In [23]:
ner_result.head()

,text,pred,pos,text_id
0,✨星空无限✨个人主页,"[星空无限, 个人主页]","[1, 6]",1
1,《野蛮女友》曝人物关系图 揭复杂故事大貌,"[野蛮女友, 人物, 关系图, 故事]","[1, 7, 9, 16]",2
2,陈楚生刚上班就遇到难缠顾客，尽责解决获前辈肯定,"[陈楚生, 上班, 遇到, 难缠, 顾客, 尽责, 解决, 前辈, 肯定]","[0, 4, 7, 9, 11, 14, 16, 19, 21]",3
3,一首《绝句》诵读千年，荣获最美成都诗词原来这么美好,"[绝句, 成都, 诗词]","[3, 15, 17]",4
4,浓香扑鼻的椰香冬阴功，汤料里放入椰汁，还有基围虾和花蛤,"[椰香冬阴功, 汤料, 椰汁, 基围虾, 花蛤]","[5, 11, 16, 21, 25]",5


In [24]:
# 包装成和train_data一样的格式
test_data = []
for index,row in ner_result.iterrows():
    item = {}
    item['text_id'] = row['text_id']
    item['text'] = row['text']
    item['mention_data'] = []
    for i,p in enumerate(row['pos']):
        item['mention_data'].append({'mention':row['pred'][i], 'offset':str(p)})
    test_data.append(item)

In [25]:
# 提取candidate候选特征
kb_column = []
train_column = []
text_id = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
mention_end = []
numerical_f = []
is_eng = []
create_works = []
end_with_point = []
end_with_line = []
end_with_line2 = []
end_with_shuminghao = []
for s in tqdm(test_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    
    for m in mention_ner:    
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['predicate']
                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            candidate_name_list = candidate_detail['alias']+[candidate_detail['subject']]
            num_alias.append(len(set(candidate_name_list)))
            m_id_list.append(m_id)
            num_candidate.append(len(candidate_ids))
            equal_subject.append(candidate_detail['subject']==m['mention'])
            text_id.append(s['text_id'])
            shu_in_abstract = False
            for sub_alia in candidate_name_list:
                if '《'+sub_alia+'》' in candidate_abstract:
                    shu_in_abstract = True
            shuminghao.append(shu_in_abstract)
#             if langid.classify(m['mention'])[0]=='en':
#                 len_mention.append(len(m['mention'].split()))
#                 is_eng.append(1)
#             else:
#                 len_mention.append(len(m['mention']))
#                 is_eng.append(0)
            len_mention.append(len(m['mention']))
            mention_start.append(m['offset'])
            mention_end.append(len(s['text'])-int(m['offset'])-len(m['mention']))
            
            
            i = 0
            for item in m_list:
                if item in candi_text: i+=1
            entity_common.append(i)
            create_works.append(is_works(m['mention'],s['text']))
            end_with_point.append('.' == s['text'][-1])
            end_with_line.append('-' in s['text'])
            end_with_line2.append('_' in s['text'])
            end_with_shuminghao.append('《' in s['text'])
        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['num_attrs'] = num_attrs
data['num_abstract_words'] = num_abstract_words
data['num_alias'] = num_alias
data['m_id'] = m_id_list
data['num_candidates'] = num_candidate 
data['mention_equal_subject'] = equal_subject
data['shuminghao'] = shuminghao
data['entity_common']  = entity_common
data['len_mention'] = len_mention
data['mention_start'] = list(map(int,mention_start))
data['create_works'] = create_works
data['mention_end'] = mention_end
data['end_with_point'] = end_with_point
data['end_with_line'] = end_with_line
data['end_with_line2'] = end_with_line2
data['end_with_shuminghao'] = end_with_shuminghao
data['works_match'] = data.apply(lambda x: 1 if x.create_works==x.shuminghao else 0,axis=1)

100%|██████████| 30000/30000 [00:10<00:00, 2999.75it/s]


In [26]:
# 提取candidate候选特征
numerical_f = []
i = 0
for s in tqdm(test_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        candidate_ids = name_id[m['mention']]

        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['predicate']
            numerical_f.append((i,candi_text,s['text']))
            i+=1
            #numerical_f.append(cal_similarity(candi_text,s['text']))


100%|██████████| 30000/30000 [00:04<00:00, 6103.71it/s]


In [27]:
import multiprocessing
p = multiprocessing.Pool(6)
res = p.map(cal_similarity, numerical_f)
res = sorted(res,key=lambda x:x[0],reverse=False)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.732 seconds.
Prefix dict has been built succesfully.
Loading model cost 0.686 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.104 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.104 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.116 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.103 seconds.
Prefix dict has been built

In [28]:
nume_f = [x[1] for x in res]
numerical_f = np.array(nume_f)
for i in range(numerical_f.shape[1]):
    data['numerical_%d'%i] = numerical_f[:,i]

In [29]:
# candidate　type的one-hot特征
data['type'] = data.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)
#data = pd.concat([data,pd.get_dummies(data['type'])],axis=1)

# target encoding
data = data.merge(groupbytype,on='type',how='left')

In [30]:
data['m_id'] = data['m_id']+max_mid+1

In [31]:
data.reset_index(drop=True,inplace=True)
data.to_pickle('data/step1_test.pkl')

In [32]:
data

,text_id,kb_id,train_mention,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,shuminghao,...,numerical_8,numerical_9,numerical_10,numerical_11,numerical_12,numerical_13,numerical_14,type,type_label_mean,type_label_count
0,1,43362,个人主页,11,156,1,257734,1,True,False,...,2.300000,0.107981,1.400000,0.065728,0.600000,0.028169,0.0,thing,0.072104,499598.0
1,2,101382,野蛮女友,12,31,1,257735,9,True,True,...,0.500000,0.138889,0.400000,0.111111,0.300000,0.083333,0.0,creativework,0.086923,249484.0
2,2,104143,野蛮女友,4,29,1,257735,9,True,True,...,0.400000,0.177778,0.250000,0.111111,0.200000,0.088889,0.0,thing,0.072104,499598.0
3,2,107628,野蛮女友,10,42,1,257735,9,True,True,...,0.850000,0.215190,0.400000,0.101266,0.300000,0.075949,0.0,creativework,0.086923,249484.0
4,2,137924,野蛮女友,24,130,6,257735,9,False,True,...,0.900000,0.056426,0.400000,0.025078,0.250000,0.015674,0.0,tvplay,0.191036,64522.0
5,2,13889,野蛮女友,13,62,1,257735,9,True,True,...,0.850000,0.161905,0.600000,0.114286,0.450000,0.085714,0.0,thing,0.072104,499598.0
6,2,16345,野蛮女友,16,76,1,257735,9,True,True,...,0.700000,0.100000,0.400000,0.057143,0.300000,0.042857,0.0,creativework,0.086923,249484.0
7,2,260196,野蛮女友,1,0,1,257735,9,True,True,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,vocabulary,0.681987,142988.0
8,2,327016,野蛮女友,3,37,1,257735,9,True,True,...,0.400000,0.170213,0.250000,0.106383,0.200000,0.085106,0.0,movie,0.083625,130476.0
9,2,71447,野蛮女友,26,109,10,257735,9,False,True,...,2.050000,0.112637,0.650000,0.035714,0.350000,0.019231,0.0,movie,0.083625,130476.0


In [5]:
import pandas as pd
test = pd.read_pickle('data/step1_test.pkl')

In [8]:
test['mention_start']

0          6
1          1
2          1
3          1
4          1
5          1
6          1
7          1
8          1
9          1
10         7
11         7
12         7
13         7
14         7
15         7
16         7
17         7
18        16
19        16
20        16
21        16
22        16
23        16
24        16
25        16
26        16
27        16
28        16
29        16
          ..
764187    20
764188    20
764189    20
764190    20
764191    20
764192    20
764193    20
764194    11
764195    11
764196    11
764197    11
764198    11
764199    11
764200    11
764201    11
764202    11
764203    11
764204    11
764205    11
764206    11
764207    11
764208    11
764209    11
764210    11
764211    11
764212    11
764213    11
764214    11
764215    11
764216    20
Name: mention_start, Length: 764217, dtype: int64